<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Artifacts_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{artifacts-basics} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{pytorch-video} -->



Use [Weights & Biases](https://wandb.com) for machine learning experiment tracking, dataset and model versioning and management, collaboration and more.

<div><img /></div>

<img src="https://wandb.me/mini-diagram" width="650" alt="Weights & Biases" />

<div><img /></div>




Use W&B Artifacts to track and version data as the inputs and outputs of your W&B Runs. In addition to logging hyperparameters, metadata, and metrics to a run, you can use an artifact to log the dataset used to train the model as input and the resulting model checkpoints as outputs.

![Artifact Simple Diagram](https://docs.wandb.ai/assets/images/artifacts_landing_page2-05443aa39ae53cede7b08908688b334a.png)

# Set Up

In order to use Weights & Biases, you will need the `wandb` package installed. You can install it as follows within Colab.

In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00


Once it is installed, the next step is to import it into your script or notebook with `import wandb`.

In [2]:
import wandb

We also need to authenticate to the Weights & Biases server. There are various ways of doing this, including for [remote or non-interactice workflows](https://docs.wandb.ai/guides/track/environment-variables), but given this is running interactively, we can use `wandb.login()`.

If we are not already authenticated, a link will appear which you can use to do so.

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Create An Dataset
Let's create some datasets that we can work with in this example.

In [5]:
import os
import numpy as np
import csv

directory = "dataset"
os.makedirs(directory, exist_ok=True)
file1, file2 = os.path.join(directory, "file1.csv"), os.path.join(directory, "file2.csv")

def generate_dummy_data(num_samples):
    data = [
        np.random.normal(50, 10, num_samples),
        np.random.randint(1, 100, num_samples),
        np.random.choice(['A', 'B', 'C', 'D'], num_samples),
        np.random.uniform(0.0, 1.0, num_samples)
    ]
    return zip(*data)

def save_to_csv(file, data):
    with open(file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['feature1', 'feature2', 'feature3', 'feature4'])
        writer.writerows(data)

num_samples = 100
save_to_csv(file1, generate_dummy_data(num_samples))
save_to_csv(file2, generate_dummy_data(num_samples))

# Create An Artifact

The general workflow for creating an Artifact is:


1.   Intialize a run.
2.   Create an Artifact.
3.   Add a any files or directories to the new Artifact that you want to track and version.
4.   Log the artifact in the W&B platform.

The most straightforward way of accomplishing this is the second line of code in the example below, which will log, track and version a new dataset (i.e. do points 2, 3, and 4 above in one step).

In [7]:
run = wandb.init(project="artifact-basics")
run.log_artifact(artifact_or_path=f"{directory}/file1.csv", name="my_first_artifact", type="dataset")
run.finish()

In the above example we first initalize a run using [`wandb.init()`](https://docs.wandb.ai/ref/python/init) the `artifact-basics` project. If this project doesn't exist, it will be created. If it alreadt exists, a new W&B Run will be added to it.


In the second line we actually log the Artifact with [`run.log_artifact()`](https://docs.wandb.ai/ref/python/public-api/run#log_artifact). In this example, we use three common arguments to the function.
1. With `artifact_or_path` we specifiy the path to where the data we want to version exists. Any file or directory can be added here.
2. with `name` we give the artifact a name within Weights & Biases that we will use to access it.
3. With `type` we give the artifact a higher level grouping. For example, we may have multiple artifacts of type data, and multiple artifacts of type model.


See the [Artifacts Reference](https://docs.wandb.ai/ref/python/artifact) guide for more information and other commonly used arguments, including how to store additional metadata.

Each time the above `log_artifact` is executed, wandb will create a new version of the Artifact within Weights & Biases if the underlying data has changed.


An alternative approach that offers more control (at the expense of more lines of code) can be seen below.

In [10]:
run = wandb.init(project="artifact-basics")

artifact = wandb.Artifact("my_first_artifact", type="dataset")
# the below will add two individual files to the artifact.
artifact.add_file(local_path=f"{directory}/file1.csv")
artifact.add_file(local_path=f"{directory}/file2.csv")
# or the below if you wanted to add the entire directory contents.
artifact.add_dir(local_path=f"{directory}")
# explictly log the artifact to Weights & Biases.
run.log_artifact(artifact)

wandb.finish()

wandb: Adding directory to artifact (./dataset)... Done. 0.0s


In the above example, lines 3-5 will create a new Artifact within your Weights & Biases project. With the resulting artifact object, you can call the [`artifact.add_file`](https://docs.wandb.ai/ref/python/artifact#add_file) or [`artifact.add_dir`](https://docs.wandb.ai/ref/python/artifact#add_dir) functions in order to add as many files and directories to the Artifact as you want. Once added, the artifact must then be explictly logged to Weights & Biases.

# Use an Artifact

When you want to use a specific version of an Artifact in a downstream task, you can specify the specific version you would like to use via either `v0`, `v1`, `v2` and so on, or via specific aliases you may have added. The `latest` alias always refers to the most recent version of the Artifact logged.

The proceeding code snippet specifies that the W&B Run will use an artifact called `my_first_artifact` with the alias `latest`:


In [11]:
run = wandb.init(project="artifact-basics")
artifact = run.use_artifact(artifact_or_name="my_first_artifact:latest") # this creates a reference within Weights & Biases that this artifact was used by this run.
path = artifact.download() # this downloads the artifact from Weights & Biases to your local system where the code is executing.
print(f"Data directory located at {path}")
run.finish()

wandb:   2 of 2 files downloaded.  


Data directory located at /content/artifacts/my_first_artifact:v1


For more information on ways to customize your Artifact download, including via the command line, see the [Download and Usage guide](https://docs.wandb.ai/guides/artifacts/download-and-use-an-artifact).

# Create a new Artifact version

Let's say we want to modify our dataset while also tracking and versioning these changes. In the below example we will subsample our dataset and save it as a new file.

In the second block of code we will log it to Weights & Biases under the same Artifact name (*my_first_artifact*) so that Weights & Biases knows that this is a new version of an existing artifact.

In [12]:
import pandas
df = pandas.read_csv(f"{directory}/file1.csv")
# subsample to 50% of the original size
df_subsampled = df.sample(frac=0.5, random_state=1)
# save the subsampled dataframe to a new file.
df_subsampled.to_csv(f"{directory}/file1.csv", index=False)

Now we have a new subsampled version of our dataset locally, we can log the new version to Weights & Biases.

In [13]:
run = wandb.init(project="artifact-basics")
run.log_artifact(artifact_or_path=f"{directory}/file1.csv", name="my_first_artifact", type="dataset", aliases =["subsampled"])
run.finish()

Now the sampled dataset will be logged to the `my_first_artifact` Artifact as a new version.

The Artifact has also been given a custom `alias`, which is a unique label for this Artifact version. While the `alias` is currently `subsampled`, the default aliases is `vN`, where `N` is the number of versions the Artifact has. This increments automatically. You can always access specific versions of an Artifact by using an alias.

# Update Artifact version metadata

You can update the `description`, `metadata`, and `alias` of an artifact on the W&B platform during or outside a W&B Run.


This example changes the `description` of the `my_first_artifact` artifact inside a run:

In [14]:
run = wandb.init(project="artifact-basics")
artifact = run.use_artifact(artifact_or_name="my_first_artifact:subsampled")
artifact.description = "This is an edited description."
artifact.metadata = {"source": "local disk", "internal data owner": "platform team"}
artifact.save()  # persists changes to an Artifact's properties
run.finish()

# Use the Artifact within your pipelines
Once the artifact is tracked and versioned within Weights & Biases it's now easy to integrate it into your ML workflows.

In [15]:
run = wandb.init(project="artifact-basics")
artifact = run.use_artifact(artifact_or_name="my_first_artifact:latest")
# the below is left as an exercise to the reader
# train model
# log model as artifact
run.finish()


# Navigate the Artifacts UI

You can also manage your Artifacts via the W&B platform. This can give you insight into your model's performance or dataset versioning. To navigate to the relevant information, click this [link](https://wandb.ai/wandb/artifact-basics/overview), then click on the **Artifacts** tab.

Navigating to the **Lineage** section in the tab will show the dependency graph formed by calling `run.use_artifact()` when an Artifact is an input to a run, and `run.log_artifact()` when an Artifact is output to a run. This helps visualize the relationship between different model versions and other objects like datasets and jobs in your project. Click [this](https://wandb.ai/wandb/artifact-basics/artifacts/dataset/my_first_artifact/v0/lineage) link to navigate to the project's lineage page.

Naturally, as you integrate W&B Artifacts into your workflow, lineage graphs such as [this interactive example](https://wandb.ai/wandb-smle/artifact_workflow/artifacts/model/quant_model/v16/lineage) will be built up over time, giving you reproducibility, governance, and auditability.

![Artifact Lineage Example](https://docs.wandb.ai/assets/images/lineage2a-e3fe54c8916c90499aaf3e1e289062bb.gif)




# Next steps
1. [Artifacts Python reference documentation](https://docs.wandb.ai/ref/python/artifact): Deep dive into artifact parameters and advanced methods.
2. [Lineage](https://docs.wandb.ai/guides/artifacts/explore-and-traverse-an-artifact-graph): View lineage graphs, which are automatically built when using W&B artifact system, providing an auditable visual overview of the relationships between specific artifact versions, datasets models and runs.
3. [Model Registry](https://docs.wandb.ai/guides/model_registry): Learn how to centralize your best artifact versions in a shared registry.
4. [Artifact Automations](https://docs.wandb.ai/guides/artifacts/project-scoped-automations): Automatically run specific Weights & Biases jobs based on changes to your artifacts, such as automatically training a new model each time a new version of the training data is logged.
5. [Reference Artifacts](https://docs.wandb.ai/guides/artifacts/track-external-files#download-a-reference-artifact): Track files saved outside the W&B server, like Amazon S3 buckets, GCS buckets, Azure blobs, and more.